In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

#### Default Paths

In [11]:
diff_expression_path = './dados_proteomica_com_foldchange.xlsx'
cancer_sheet = 'Cancer-Rat'
preeclam_sheet = 'Preeclampsia-Rat'

string_cancer_path = './string_cancer.csv'
string_preeclam_path = './String_pre_eclampsia.csv'

kegg_cancer_path = './KEGG_cancer.csv'
kegg_preeclam_path = './KEGG_pre_eclampsia.xlsx'

output_path = Path('./output')
output_path.mkdir(exist_ok=True)

#### Load expression data

In [28]:
cancer_exp_cols = ['Query_protein', 'Gene_name', 'FC']
rename_cancer_exp_cols = {'Query_protein':'protein', 'Gene_name':'gene', 'FC':'fc'}
cancer_expression_data = pd.read_excel(diff_expression_path, sheet_name=cancer_sheet, usecols=cancer_exp_cols).rename(columns=rename_cancer_exp_cols)
cancer_expression_data['regulated'] = np.where(cancer_expression_data['fc'] > 0, 'up', 'down')

preeclam_exp_cols = ['protein_firstname', 'Gene_name_correct', 'FC']
rename_preeclam_exp_cols = {'protein_firstname':'protein', 'Gene_name_correct':'gene', 'FC':'fc'}
preeclam_expression_data = pd.read_excel(diff_expression_path, sheet_name=preeclam_sheet, usecols=preeclam_exp_cols).rename(columns=rename_preeclam_exp_cols)
preeclam_expression_data['regulated'] = np.where(preeclam_expression_data['fc'] > 0, 'up', 'down')


In [29]:
# Drop rows missing gene or fold change

print('## Cancer NAs:\n', cancer_expression_data.isna().sum())
print()
print('## Pre-eclam NAs:\n', preeclam_expression_data.isna().sum())

cancer_expression_data.dropna(how='any', subset=['gene', 'fc'], inplace=True)
preeclam_expression_data.dropna(how='any', subset=['gene', 'fc'], inplace=True)

## Cancer NAs:
 protein      0
gene         0
fc           1
regulated    0
dtype: int64

## Pre-eclam NAs:
 protein       0
gene         10
fc            0
regulated     0
dtype: int64


In [5]:
# Cancer - Drop duplicated genes (?) (keep the first occurrence)
display(cancer_expression_data['gene'].value_counts()[cancer_expression_data['gene'].value_counts() > 1])
display(cancer_expression_data[cancer_expression_data['gene'] == 'Tuba1c'])
cancer_expression_data.drop_duplicates(subset='gene', keep='first', inplace=True)

Tuba1c    2
Name: gene, dtype: int64

,protein,gene,fc,regulated
0,Q6AYZ1,Tuba1c,-0.440370,down
160,A0A0H2UHM7,Tuba1c,-0.355256,down


In [6]:
# Pre-eclampsia - Drop duplicated genes (?) (keep the first occurrence)
display(preeclam_expression_data['gene'].value_counts()[preeclam_expression_data['gene'].value_counts() > 1])
display(preeclam_expression_data[(preeclam_expression_data['gene'] == 'Gbe1') | (preeclam_expression_data['gene'] == 'Gpx3')])
preeclam_expression_data.drop_duplicates(subset='gene', keep='first', inplace=True)

Gbe1    2
Gpx3    2
Name: gene, dtype: int64

,protein,gene,fc,regulated
208,A0A0G2JTB2,Gbe1,0.533101,up
209,A0A096MJY6,Gbe1,0.537608,up
317,A0A0G2K531,Gpx3,-0.320739,down
318,A0A0G2K8W9,Gpx3,-0.290763,down


##### Merge differential expression data (cancer + pre-eclampsia)

In [7]:
merged_expression_data = cancer_expression_data.merge(preeclam_expression_data, on='gene', suffixes=['_cancer', '_pre_eclampsia'], how='outer')
merged_expression_data['cancer'] = np.where(merged_expression_data['fc_cancer'].notna(), 1, 0)
merged_expression_data['pre_eclampsia'] = np.where(merged_expression_data['fc_pre_eclampsia'].notna(), 1, 0)
merged_expression_data['both'] = np.where((merged_expression_data['cancer'] == 1) & (merged_expression_data['pre_eclampsia'] == 1), 1, 0)


In [8]:
merged_expression_data.head(10)

,protein_cancer,gene,fc_cancer,regulated_cancer,protein_pre_eclampsia,fc_pre_eclampsia,regulated_pre_eclampsia,cancer,pre_eclampsia,both
0,Q6AYZ1,Tuba1c,-0.440370,down,NaN,NaN,NaN,1,0,0
1,Q9EPH1,A1bg,-0.778146,down,NaN,NaN,NaN,1,0,0
2,P06238,A2m,-0.711127,down,NaN,NaN,NaN,1,0,0
3,G3V9J6,Abcb1b,-0.557322,down,NaN,NaN,NaN,1,0,0
4,P68136,Acta1,-0.284675,down,NaN,NaN,NaN,1,0,0
5,P62738,Acta2,-0.316151,down,NaN,NaN,NaN,1,0,0
6,D3ZRN3,Actbl2,-0.322669,down,NaN,NaN,NaN,1,0,0
7,P68035,Actc1,-0.284966,down,NaN,NaN,NaN,1,0,0
8,P63259,Actg2,-0.308877,down,NaN,NaN,NaN,1,0,0
9,Q9Z1P2,Actn1,0.240738,up,Q9Z1P2,0.263762,up,1,1,1


In [9]:
# Verify common genes
merged_expression_data[merged_expression_data['fc_cancer'].notna() & merged_expression_data['fc_pre_eclampsia'].notna()]

,protein_cancer,gene,fc_cancer,regulated_cancer,protein_pre_eclampsia,fc_pre_eclampsia,regulated_pre_eclampsia,cancer,pre_eclampsia,both
9,Q9Z1P2,Actn1,0.240738,up,Q9Z1P2,0.263762,up,1,1,1
48,Q6TUG0,Dnajb11,-0.428988,down,Q6TUG0,0.442867,up,1,1,1
51,Q641Z6,Ehd1,-0.450791,down,Q641Z6,-0.294619,down,1,1,1
52,Q8R3Z7,Ehd4,0.576265,up,Q8R3Z7,-0.293523,down,1,1,1
60,G3V843,F2,-0.553859,down,G3V843,0.293442,up,1,1,1
76,P20059,Hpx,0.444360,up,P20059,0.555551,up,1,1,1
89,P11762,Lgals1,-0.287505,down,P11762,0.315815,up,1,1,1
117,Q9R063,Prdx5,-0.360268,down,A0A0G2JSS8,-0.353523,down,1,1,1
141,G3V8T5,Ruvbl2,-0.296228,down,G3V8T5,0.217608,up,1,1,1
148,Q794F9,Slc3a2,-0.531778,down,Q794F9,-0.290488,down,1,1,1


In [32]:
# Count number of genes
ct_cancer = sum(merged_expression_data['cancer'] == 1)
ct_preeclam = sum(merged_expression_data['pre_eclampsia'] == 1)
ct_both = sum(merged_expression_data['both'] == 1)
print(f'# Cancer = {ct_cancer}, # Pre-eclampsia = {ct_preeclam}, # Common = {ct_both}')


# Cancer = 175, # Pre-eclampsia = 314, # Common = 12


In [19]:
# Save data
exp_data_fn = 'expression_data'
cancer_expression_data.to_csv(Path(output_path, f'{exp_data_fn}_cancer.csv'), index=False)
preeclam_expression_data.to_csv(Path(output_path, f'{exp_data_fn}_pre_eclampsia.csv'), index=False)
merged_expression_data.to_csv(Path(output_path, f'{exp_data_fn}_merged.csv'), index=False)

#### Load String-db data (PPI)

In [77]:
string_cols = ['#node1', 'node2', 'node1_string_id', 'node2_string_id']
rename_string_cols = {'#node1': 'node1', 'node1_string_id':'node1_id', 'node2_string_id':'node2_id'}

# Note: current string files are separated by semicolon
cancer_ppi_data = pd.read_csv(string_cancer_path, usecols=string_cols, sep=';').rename(columns=rename_string_cols)
preeclam_ppi_data = pd.read_csv(string_preeclam_path, usecols=string_cols, sep=';').rename(columns=rename_string_cols)

In [80]:
display(cancer_ppi_data.head(10))

cancer_genes_in_ppi = set(cancer_ppi_data['node1'].values.tolist() + cancer_ppi_data['node2'].values.tolist())
cancer_all_genes = set(cancer_expression_data['gene'].values.tolist())
print(f'Cancer -> # Total genes = {len(cancer_all_genes)}, # Genes in PPI = {len(cancer_genes_in_ppi)}')

print(f'Genes NOT in ppi ({len(cancer_all_genes - cancer_genes_in_ppi)}):')
print(cancer_all_genes - cancer_genes_in_ppi)


,node1,node2,node1_id,node2_id
0,Acta1,Actc1,10116.ENSRNOP00000024084,10116.ENSRNOP00000011773
1,Acta1,Acta2,10116.ENSRNOP00000024084,10116.ENSRNOP00000073101
2,Acta1,Actg2,10116.ENSRNOP00000024084,10116.ENSRNOP00000050322
3,Acta2,Actc1,10116.ENSRNOP00000073101,10116.ENSRNOP00000011773
4,Acta2,Tln1,10116.ENSRNOP00000073101,10116.ENSRNOP00000022401
5,Acta2,Pxn,10116.ENSRNOP00000073101,10116.ENSRNOP00000043928
6,Acta2,Actg2,10116.ENSRNOP00000073101,10116.ENSRNOP00000050322
7,Acta2,Myl6,10116.ENSRNOP00000073101,10116.ENSRNOP00000070200
8,Acta2,Myh14,10116.ENSRNOP00000073101,10116.ENSRNOP00000072636
9,Acta2,Vcl,10116.ENSRNOP00000073101,10116.ENSRNOP00000074748


Cancer -> # Total genes = 175, # Genes in PPI = 104
Genes NOT in ppi (71):
{'Ubap2l', 'Dstn', 'Anxa2', 'Vat1', 'Alpp', 'Rpia', 'Rap2c', 'Tubb4b', 'Prmt8', 'Cyb5b', 'Afm', 'Bsg', 'Atic', 'Psap', 'Ddost', 'Lgals1', 'Cdv3', 'Mug2', 'Ndrg1', 'Emc2', 'Lonp1', 'Car1', 'Atxn10', 'Slc25a10', 'Cntrl', 'Ddx17', 'Hpx', 'Etfb', 'Rilpl1', 'Cpsf6', 'Nefl', 'Pfkl', 'Stk26', 'Serpina3n', 'Aldh2', 'Rab33b', 'Dpysl2', 'Csrp1', 'Rragb', 'Dbnl', 'Aimp1', 'Gstm4', 'Prdx4', 'Fkbp14', 'Uchl4', 'Abcb1b', 'Atp1b1', 'Hbb-b2', 'Mat2a', 'Hax1', 'Cand1', 'Hba', 'Rab43', 'Itih2', 'Spata1', 'A1bg', 'Api5', 'A2m', 'En2', 'Ak2', 'Ptbp1', 'Olfm4', 'Cotl1', 'Tgm2', 'Gnpda1', 'Sfxn3', 'Mug1', 'Rbp4', 'Slc3a2', 'Vapa', 'Krt18'}


In [81]:
display(preeclam_ppi_data.head(10))

preeclam_genes_in_ppi = set(preeclam_ppi_data['node1'].values.tolist() + preeclam_ppi_data['node2'].values.tolist())
preeclam_all_genes = set(preeclam_expression_data['gene'].values.tolist())
print(f'Cancer -> # Total genes = {len(preeclam_all_genes)}, # Genes in PPI = {len(preeclam_genes_in_ppi)}')

print(f'Genes NOT in ppi ({len(preeclam_all_genes - preeclam_genes_in_ppi)}):')
print(preeclam_all_genes - preeclam_genes_in_ppi)

,node1,node2,node1_id,node2_id
0,Actn1,Itga6,10116.ENSRNOP00000068851,10116.ENSRNOP00000002075
1,Actn1,Mcam,10116.ENSRNOP00000068851,10116.ENSRNOP00000010464
2,Actn1,Efhd2,10116.ENSRNOP00000068851,10116.ENSRNOP00000018864
3,Actn1,Itga2b,10116.ENSRNOP00000068851,10116.ENSRNOP00000052051
4,Ada,Nt5c3a,10116.ENSRNOP00000014151,10116.ENSRNOP00000071357
5,Ada,Adk,10116.ENSRNOP00000014151,10116.ENSRNOP00000073983
6,Adk,Ak2,10116.ENSRNOP00000073983,10116.ENSRNOP00000000134
7,Adk,Jchain,10116.ENSRNOP00000073983,10116.ENSRNOP00000004866
8,Adk,Upp1,10116.ENSRNOP00000073983,10116.ENSRNOP00000006765
9,Adk,Nt5c3a,10116.ENSRNOP00000073983,10116.ENSRNOP00000071357


Cancer -> # Total genes = 314, # Genes in PPI = 153
Genes NOT in ppi (200):
{'Uggt1 ', 'Abcb4', 'Snx12', 'Kif2a ', 'Etfdh ', 'Slc40a1', 'LOC102557244', 'Pappa2', 'Cmas', 'Stmn1', 'Plvap', 'Mcam ', 'Matr3', 'Tspan9', 'Tfg ', 'Prl3d4', 'Krt19 ', 'Nt5c3a ', 'RGD1562844', 'Cox5b', 'Hsd17b11 ', 'Camk2d', 'Psg29 ', 'Uso1 ', 'Lad1', 'Man2a1', 'Tram2', 'Npc2', 'Eps8l2', 'Serpinb9', 'Pdxk', 'Tes', 'St6gal1 ', 'Tnfaip8', 'Pdlim4 ', 'Pltp', 'Fam122b', 'Pecam', 'RT1-CE1 ', 'Lman1 ', 'Grn', 'Ctla2a', 'Ccdc51', 'Lgalsl', 'Prxl2a ', 'Tbl2 ', 'C4bpa', 'Pgk1 ', 'Hyou1 ', 'Ppl', 'Slc2a1', 'Gab1', 'Atp2b1', 'Pon2', 'Emb ', 'Creld2', 'Aldh2 ', 'Myo18a', 'Phactr4', 'Lgals5', 'Nostrin', 'Psg19', 'Sbsn', 'Rps15 ', 'Crtap ', 'Pitrm1', 'LOC299282', 'Dpp3', 'Xdh', 'Ckap4', 'Prl6a1', 'Fblim1', 'Pcmt1', 'Nln', 'Hpx', 'Hmgb3 ', 'Nars1 ', 'Ak2 ', 'Prl4a1', 'Colec12 ', 'Csrp2', 'Sgpl1 ', 'Naxd ', 'Inf2', 'Cpne1', 'Sec11a ', 'Kng1', 'Mustn1', 'Lrrc59 ', 'Hp', 'Orm1', 'Timp3', 'Lima1 ', 'Agt ', 'Sptan1 ', 'Ces1c ', 'U

In [82]:
###### Verificar como podem existir Genes no STRING que não estão na tabela de expressão diferencial
preeclam_ppi_data = preeclam_ppi_data[preeclam_ppi_data['node1'].isin(preeclam_all_genes) & preeclam_ppi_data['node2'].isin(preeclam_all_genes)]



preeclam_genes_in_ppi = set(preeclam_ppi_data['node1'].values.tolist() + preeclam_ppi_data['node2'].values.tolist())
preeclam_all_genes = set(preeclam_expression_data['gene'].values.tolist())
print(f'Cancer -> # Total genes = {len(preeclam_all_genes)}, # Genes in PPI = {len(preeclam_genes_in_ppi)}')

print(f'Genes NOT in ppi ({len(preeclam_all_genes - preeclam_genes_in_ppi)}):')
print(preeclam_all_genes - preeclam_genes_in_ppi)

Cancer -> # Total genes = 314, # Genes in PPI = 106
Genes NOT in ppi (208):
{'Uggt1 ', 'Abcb4', 'Snx12', 'Kif2a ', 'Etfdh ', 'Slc40a1', 'LOC102557244', 'Pappa2', 'Cmas', 'Stmn1', 'Plvap', 'Mcam ', 'Matr3', 'Tspan9', 'Tfg ', 'Prl3d4', 'Krt19 ', 'Nt5c3a ', 'RGD1562844', 'Cox5b', 'Hsd17b11 ', 'Camk2d', 'Psg29 ', 'Uso1 ', 'Lad1', 'Man2a1', 'Erp29', 'Tram2', 'Npc2', 'Eps8l2', 'Serpinb9', 'Pdxk', 'Tes', 'St6gal1 ', 'Tnfaip8', 'Pdlim4 ', 'Pltp', 'Fam122b', 'Pecam', 'RT1-CE1 ', 'Lman1 ', 'Grn', 'Ctla2a', 'Ccdc51', 'Lgalsl', 'Prxl2a ', 'Tbl2 ', 'C4bpa', 'Pgk1 ', 'Hyou1 ', 'Ppl', 'Slc2a1', 'Gab1', 'Atp2b1', 'Pon2', 'Emb ', 'Creld2', 'Aldh2 ', 'Myo18a', 'Phactr4', 'Lgals5', 'Nostrin', 'Psg19', 'Sbsn', 'Rps15 ', 'Crtap ', 'Pitrm1', 'LOC299282', 'Dpp3', 'Xdh', 'Ckap4', 'Prl6a1', 'Fblim1', 'Pcmt1', 'Nln', 'Hpx', 'Hmgb3 ', 'Nars1 ', 'Ak2 ', 'Prl4a1', 'Colec12 ', 'Csrp2', 'Sgpl1 ', 'Naxd ', 'Inf2', 'Cpne1', 'Sec11a ', 'Kng1', 'Mustn1', 'Lrrc59 ', 'Hp', 'Orm1', 'Timp3', 'Lima1 ', 'Agt ', 'Sptan1 ', 'Ce

In [83]:
# Merge STRING networks and check/remove for duplicates
merged_ppi_data = pd.concat([cancer_ppi_data, preeclam_ppi_data], axis=0, join='outer', ignore_index=True)
display(merged_ppi_data)
print()
merged_ppi_data.pivot_table(index = ['node1', 'node2'], aggfunc ='size')[merged_ppi_data.pivot_table(index = ['node1', 'node2'], aggfunc ='size') > 1]

,node1,node2,node1_id,node2_id
0,Acta1,Actc1,10116.ENSRNOP00000024084,10116.ENSRNOP00000011773
1,Acta1,Acta2,10116.ENSRNOP00000024084,10116.ENSRNOP00000073101
2,Acta1,Actg2,10116.ENSRNOP00000024084,10116.ENSRNOP00000050322
3,Acta2,Actc1,10116.ENSRNOP00000073101,10116.ENSRNOP00000011773
4,Acta2,Tln1,10116.ENSRNOP00000073101,10116.ENSRNOP00000022401
...,...,...,...,...
354,Rps9,Tmed3,10116.ENSRNOP00000074779,10116.ENSRNOP00000018603
355,Ruvbl2,Sdf2l1,10116.ENSRNOP00000028217,10116.ENSRNOP00000002540
356,Ruvbl2,Uchl5,10116.ENSRNOP00000028217,10116.ENSRNOP00000072164
357,Sec61a1,Sec63,10116.ENSRNOP00000018455,10116.ENSRNOP00000069126


node1    node2 
Dnajb11  Ruvbl2    2
Ehd1     Ehd4      2
dtype: int64

In [84]:
merged_ppi_data.drop_duplicates(keep='first', ignore_index=True, inplace=True)
unique_nodes = set(preeclam_ppi_data['node1'].values.tolist() + preeclam_ppi_data['node2'].values.tolist())
print(f'# of Edges = {len(merged_ppi_data)}, # of Nodes = {len(unique_nodes)}')


# of Edges = 357, # of Nodes = 106


In [85]:
ppi_fn = 'ppi_data'
cancer_ppi_data.to_csv(Path(output_path, f'./{ppi_fn}_cancer.csv'), index=False)
preeclam_ppi_data.to_csv(Path(output_path, f'./{ppi_fn}_pre_eclampsia.csv'), index=False)
merged_ppi_data.to_csv(Path(output_path, f'./{ppi_fn}_merged.csv'), index=False)


Load and process KEGG enrichment (pathways)

In [13]:
pathway_rename_cols = {
    '#term ID' : 'pathway_id',
    'term description' : 'pathway_name',
    'observed gene count' : 'observed_gene_count',
    'background gene count' : 'background_gene_count',
    'strength' : 'strength',
    'false discovery rate' : 'fdr',
    'matching proteins in your network (labels)' : 'gene_labels_in'
}


In [14]:
cancer_pathway_data = pd.read_csv(kegg_cancer_path, sep=';', usecols=list(pathway_rename_cols.keys())).rename(columns=pathway_rename_cols)
cancer_pathway_net = cancer_pathway_data[['pathway_id', 'pathway_name', 'gene_labels_in']].copy()
cancer_pathway_net['gene_labels_in'] = cancer_pathway_net['gene_labels_in'].str.split(',')
cancer_pathway_net = cancer_pathway_net.explode(column='gene_labels_in', ignore_index=True)
cancer_pathway_net

,pathway_id,pathway_name,gene_labels_in
0,rno04145,Phagosome,Tfrc
1,rno04145,Phagosome,Tuba4a
2,rno04145,Phagosome,Tubb3
3,rno04145,Phagosome,Tubb6
4,rno04145,Phagosome,Ctsl
...,...,...,...
112,rno04144,Endocytosis,Ap2b1
113,rno01230,Biosynthesis of amino acids,Pfkl
114,rno01230,Biosynthesis of amino acids,Rpia
115,rno01230,Biosynthesis of amino acids,Mat2a


In [15]:
preeclam_pathway_data = pd.read_excel(kegg_preeclam_path, usecols=list(pathway_rename_cols.keys())).rename(columns=pathway_rename_cols)
preeclam_pathway_net = preeclam_pathway_data[['pathway_id', 'pathway_name', 'gene_labels_in']].copy()
preeclam_pathway_net['gene_labels_in'] = preeclam_pathway_net['gene_labels_in'].str.split(',')
preeclam_pathway_net = preeclam_pathway_net.explode(column='gene_labels_in', ignore_index=True)
preeclam_pathway_net

,pathway_id,pathway_name,gene_labels_in
0,rno04141,Protein processing in endoplasmic reticulum,Hsph1
1,rno04141,Protein processing in endoplasmic reticulum,Erp29
2,rno04141,Protein processing in endoplasmic reticulum,Dnajb11
3,rno04141,Protein processing in endoplasmic reticulum,Sec31a
4,rno04141,Protein processing in endoplasmic reticulum,Calr
...,...,...,...
184,rno04922,Glucagon signaling pathway,Slc2a1
185,rno04216,Ferroptosis,Tfrc
186,rno04216,Ferroptosis,Slc40a1
187,rno04216,Ferroptosis,Hmox1


In [16]:
# Merge both pathway nets and check/remove for duplicates

merged_pathway_net = pd.concat([cancer_pathway_net, preeclam_pathway_net], axis=0, join='outer', ignore_index=True)
display(merged_pathway_net)
print()
merged_pathway_net.pivot_table(index = ['pathway_name', 'gene_labels_in'], aggfunc ='size')[merged_pathway_net.pivot_table(index = ['pathway_name', 'gene_labels_in'], aggfunc ='size') > 1]

,pathway_id,pathway_name,gene_labels_in
0,rno04145,Phagosome,Tfrc
1,rno04145,Phagosome,Tuba4a
2,rno04145,Phagosome,Tubb3
3,rno04145,Phagosome,Tubb6
4,rno04145,Phagosome,Ctsl
...,...,...,...
301,rno04922,Glucagon signaling pathway,Slc2a1
302,rno04216,Ferroptosis,Tfrc
303,rno04216,Ferroptosis,Slc40a1
304,rno04216,Ferroptosis,Hmox1


pathway_name                                 gene_labels_in
Phagosome                                    Tfrc              2
                                             Tuba4a            2
                                             Tubb3             2
Protein processing in endoplasmic reticulum  Dnajb11           2
                                             Ero1a             2
                                             Lman2             2
dtype: int64

In [17]:
merged_pathway_net.drop_duplicates(keep='first', ignore_index=True, inplace=True)

In [18]:
pathway_net_fn = 'kegg_net'
cancer_pathway_net.to_csv(Path(output_path, f'./{pathway_net_fn}_cancer.csv'), index=False)
preeclam_pathway_net.to_csv(Path(output_path, f'./{pathway_net_fn}_pre_eclampsia.csv'), index=False)
merged_pathway_net.to_csv(Path(output_path, f'./{pathway_net_fn}_merged.csv'), index=False)